In [1]:
import io
import random
import string # para procesar cadenas de python estándar
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # para descargar los paquetes más populares/comunes
#nltk.download('punkt') # descargo punkt
#nltk.download('wordnet') # descargo wordnet

True

In [3]:
help(nltk) #visualizo todos los paquetes que tiene nltk

Help on package nltk:

NAME
    nltk

DESCRIPTION
    The Natural Language Toolkit (NLTK) is an open source Python library
    for Natural Language Processing.  A free online book is available.
    (If you use the library for academic research, please cite the book.)
    
    Steven Bird, Ewan Klein, and Edward Loper (2009).
    Natural Language Processing with Python.  O'Reilly Media Inc.
    http://nltk.org/book
    
    @version: 3.4.5

PACKAGE CONTENTS
    app (package)
    book
    ccg (package)
    chat (package)
    chunk (package)
    classify (package)
    cluster (package)
    collections
    collocations
    compat
    corpus (package)
    data
    decorators
    downloader
    draw (package)
    featstruct
    grammar
    help
    inference (package)
    internals
    jsontags
    lazyimport
    lm (package)
    metrics (package)
    misc (package)
    parse (package)
    probability
    sem (package)
    sentiment (package)
    stem (package)
    tag (package)
    tbl (pac

In [4]:
nltk.__version__ # version de nltk

'3.4.5'

Leeremos en el archivo corpus.txt y convertiremos el corpus completo en una lista de oraciones y una lista de palabras para un procesamiento previo adicional.

In [5]:
f = open('chatbot.txt','r', errors = 'ignore')
raw = f.read()
raw = raw.lower()   #se convierte a minúsculas

#TOKENIZACIÓN

sent_tokens = nltk.sent_tokenize(raw)  #se convierte en una lista de oraciones
word_tokens = nltk.word_tokenize(raw)  #se convierte en una lista de palabras

In [6]:
# Veamos un ejemplo de sent_tokens

sent_tokens[:2]

['a chatbot is a software application used to conduct an on-line chat conversation via text or text-to-speech, in lieu of providing direct contact with a live human agent.designed to convincingly simulate the way a human would behave as a conversational partner, chatbot systems typically require continuous tuning and testing, and many in production remain unable to adequately converse or pass the industry standard turing test.the term "chatterbot" was originally coined by michael mauldin (creator of the first verbot) in 1994 to describe these conversational programs.',
 'chatbots are typically used in dialog systems for various purposes including customer service, request routing, or for information gathering.']

In [7]:
# Veamos un ejemplo de word_tokens

word_tokens[:2]

['a', 'chatbot']

*Pre-procesamiento del texto crudo*

Ahora definiremos una función llamada LemTokens que tomará como entrada los tokens y devolverá tokens normalizados.

In [8]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet es un diccionario de inglés semánticamente incluido en NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

*Coincidencia de palabras clave*

A continuación, definiremos una función para un saludo a través el bot, es decir, si la entrada de un usuario es un saludo, el bot devolverá una respuesta de saludo. 

In [9]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

*Generando respuesta*

Para generar una respuesta de nuestro bot para preguntas de entrada, se utilizará el concepto de similitud de documentos. Entonces comenzamos importando los módulos necesarios.
Desde la biblioteca scikit learn, importaremos el vectorizador TFidf para convertir una colección de documentos en bruto en una matriz de características TF-IDF.
Además, importaremos el módulo de similitud de coseno desde la biblioteca de aprendizaje de scikit

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Esto se usará para encontrar la similitud entre las palabras ingresadas por el usuario y las palabras en el corpus. Esta es la implementación más simple posible de un chatbot.

Definimos una respuesta de función que busca la expresión del usuario para una o más palabras clave conocidas y devuelve una de las varias repsuestas posibles. Si no encuentra la entrada que coincide con ninguna de las palabras clave, devuelve una respuesta: “¡Lo siento! No te entiendo.

In [11]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

Finalmente, alimentaremos las líneas que queremos que diga nuestro robot al iniciar y finalizar una conversación, según la información del usuario.

In [12]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
hi
ROBO: *nods*
who is Alan Turing
ROBO: in 1950, alan turing's famous article "computing machinery and intelligence" was published, which proposed what is now called the turing test as a criterion of intelligence.
what is ELIZA?
ROBO: among the most notable early chatbots are eliza (1966) and parry (1972).
you are really thinking?
ROBO: I am sorry! I don't understand you
jum! which are Limitations of Chatbots
ROBO: limitations of chatbots
the creation and implementation of chatbots is still a developing area, heavily related to artificial intelligence and machine learning, so the provided solutions, while possessing obvious advantages, have some important limitations in terms of functionalities and use cases.
Ok. Bye!
ROBO: I am sorry! I don't understand you
Bye
ROBO: Bye! take care..
